In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from copy import deepcopy
from matplotlib import pyplot as plt

import os
import datetime
import time
import json
import pickle
import os
import ast

NAFILL = -999

In [ ]:
with open("config_general.json", "r") as f:
    config_general = json.load(f)
folder_path = config_general["data_root_path"]
epis = config_general["epis"]
wks = config_general["wks"]

In [ ]:
config_specific_filename = "config_specific_cmu_two_non_overlap.json"
with open(config_specific_filename, "r") as f:
    config_specific = json.load(f)
print(config_specific)
focus_path = folder_path + "Selection_Group/" + config_specific["label_focus"] + "/"
if (not os.path.exists(focus_path)):
    os.mkdir(focus_path)

In [ ]:
def rule_filter(df, minsupp, minconf, minlift = -1, lift_valid = True):
    df_filtered = deepcopy(df)
    if (lift_valid):
        df_filtered = df_filtered[
            (
                (
                (df_filtered["supp" + '_grp1'] >= minsupp) &
                (df_filtered["conf" + '_grp1'] >= minconf) &
                (df_filtered["lift" + '_grp1'] >= minlift)
                ) 
                |
                (
                (df_filtered["supp" + '_grp2'] >= minsupp) &
                (df_filtered["conf" + '_grp2'] >= minconf) &
                (df_filtered["lift" + '_grp2'] >= minlift)
                )
            )]
    else:
        df_filtered = df_filtered[
            (
                (
                (df_filtered["supp" + '_grp1'] >= minsupp) &
                (df_filtered["conf" + '_grp1'] >= minconf)
                ) 
                |
                (
                (df_filtered["supp" + '_grp2'] >= minsupp) &
                (df_filtered["conf" + '_grp2'] >= minconf)
                )
            )]
    return df_filtered

In [ ]:
def nostraight_pair_rulenum(df):
    return sum((df["idx_grp1"] == 0) | (df["idx_grp2"] == 0))
def nostraight_paired(df):
    return df[(df["idx_grp1"] == 0) | (df["idx_grp2"] == 0)]
def straight_paired(df):
    return df[(df["idx_grp1"] != 0) & (df["idx_grp2"] != 0)]

In [ ]:
dfs_combined_asso = {}
for wk in wks:
    dfs_combined_asso[wk] = {}
    for epi in epis:
        print(wk, epi)
        filename = "asso_" + wk + "_" + epi
#         df_grp1 = pd.read_excel(grp_spmf_parse + filename + "_grp1.xlsx")
#         df_grp2 = pd.read_excel(grp_spmf_parse + filename + "_grp2.xlsx")
        df_combine = pd.read_csv(focus_path + "SPMF_parse/" + filename + "_combine.csv")
        dfs_combined_asso[wk][epi] = deepcopy(df_combine)

In [ ]:
dfs_filtered_asso = {}
dfs_filtered_asso_straight = {}
dfs_filtered_asso_nostraight = {}
for wk in wks:
    dfs_filtered_asso[wk] = {}
    dfs_filtered_asso_straight[wk] = {}
    dfs_filtered_asso_nostraight[wk] = {}
    for epi in epis:
        print(wk, epi)
        df_combined = deepcopy(dfs_combined_asso[wk][epi])
        print(df_combined.shape)
        th_delta = (straight_paired(df_combined)[["supp_diff","conf_diff","lift_diff"]].abs().quantile(0.99)).tolist()
        df_filtered = rule_filter(df_combined,
                                  config_specific["thresholds_for_rule_mining"][wk][epi][0] + th_delta[0],
                                  config_specific["thresholds_for_rule_mining"][wk][epi][1] + th_delta[1],
                                  config_specific["thresholds_for_rule_mining"][wk][epi][2] + th_delta[2])
        dfs_filtered_asso[wk][epi] = deepcopy(df_filtered)
        dfs_filtered_asso_straight[wk][epi] = deepcopy(straight_paired(df_filtered))
        dfs_filtered_asso_nostraight[wk][epi] = deepcopy(nostraight_paired(df_filtered))
        print(dfs_filtered_asso_nostraight[wk][epi].shape)
        print(nostraight_pair_rulenum(df_filtered))

In [ ]:
def remove_close_rules(ruleset):
    Xs = ruleset["X"].apply(lambda x : ast.literal_eval(x))
    Ys = ruleset["Y"].apply(lambda x : ast.literal_eval(x))
    idx_to_remove = []
    idx_parent = []
    for idx, X, Y in zip(range(ruleset.shape[0]), Xs, Ys):
        for idxx, X_, Y_ in zip(range(ruleset.shape[0]), Xs, Ys):
            if ((set(X) < set(X_) and set(Y) <= set(Y_))
               or 
               (set(X) <= set(X_) and set(Y) < set(Y_))):
                idx_to_remove.append(idx)
                idx_parent.append(idxx)
                break
    return ruleset.drop(ruleset.index[idx_to_remove])

## Rule pair - straight

In [ ]:
top_k = 10
w1 = 1.0
w2 = 1.5
w3 = 0.5

In [ ]:
if (not os.path.exists(focus_path + "SPMF_select/")):
    os.mkdir(focus_path + "SPMF_select/")

In [ ]:
rulesets_asso_straight = {}
for wk in wks:
    rulesets_asso_straight[wk] = {}
    for epi in epis:
        df_filtered = deepcopy(dfs_filtered_asso_straight[wk][epi])
        # for row_idx, row_filter in deepcopy(df_filtered).iterrows():
        df_filtered["p_y1"] = df_filtered["conf_grp1"] / df_filtered["lift_grp1"]
        df_filtered["p_y2"] = df_filtered["conf_grp2"] / df_filtered["lift_grp2"]
        df_filtered["p_x1"] = df_filtered["supp_grp1"] / df_filtered["conf_grp1"]
        df_filtered["p_x2"] = df_filtered["supp_grp2"] / df_filtered["conf_grp2"]

        X_len = df_filtered["X"].apply(lambda x : len(ast.literal_eval(x)))
        df_filtered["delta_p_x"] = df_filtered["p_x1"] - df_filtered["p_x2"]
        delta_supp_1 = 2 * df_filtered["supp_diff"] / (df_filtered["p_y1"] + df_filtered["p_y2"])
        delta_supp_2 = (df_filtered["p_y2"] - df_filtered["p_y1"]) * (df_filtered["supp_grp1"] + df_filtered["supp_grp2"]) / (df_filtered["p_y1"] + df_filtered["p_y2"])
        df_filtered["supp_delta"] = delta_supp_1 + delta_supp_2
        
        df_filtered["new_three_weight"] = np.sign(df_filtered["delta_p_x"]) * np.sign(df_filtered["conf_diff"]) *\
                                          np.power(X_len,w1) * \
                                          np.power(np.abs(df_filtered["delta_p_x"]),w2) * \
                                          np.power(np.abs(df_filtered["conf_diff"]),w3)
        ruleset1 = deepcopy(df_filtered.sort_values(by = ["new_three_weight"], ascending=False).head(top_k))

        ruleset_asso_straight = pd.concat([ruleset1])

        ruleset_asso_straight.drop_duplicates(inplace=True)
        print("remove {} duplicates".format(ruleset1.shape[0] - ruleset_asso_straight.shape[0]))
        ruleset_asso_straight_ = deepcopy(remove_close_rules(ruleset_asso_straight))
        print("remove {} closeset".format(ruleset_asso_straight.shape[0] - ruleset_asso_straight_.shape[0]))
        ruleset_asso_straight = deepcopy(ruleset_asso_straight_)
        print("final shape", ruleset_asso_straight.shape)

        rulesets_asso_straight[wk][epi] = deepcopy(ruleset_asso_straight)
        ruleset_asso_straight.to_csv(focus_path + "SPMF_select/" + "asso_rule_straight" + "_" + wk + "_" + epi + ".csv", index = False)

## Rule pair - not-straight

In [ ]:
rulesets_asso_nostraight = {}
for wk in wks:
    rulesets_asso_nostraight[wk] = {}
    for epi in epis:
        df_filtered = deepcopy(dfs_filtered_asso_nostraight[wk][epi])
        # for row_idx, row_filter in deepcopy(df_filtered).iterrows():
        df_filtered["p_x1"] = 0
        df_filtered["p_x2"] = 0
        df_filtered["p_y1"] = 0
        df_filtered["p_y2"] = 0

        flag_grp1 = df_filtered["idx_grp1"] == 0
        df_filtered["p_y1"][flag_grp1] = 0
        df_filtered["p_y2"][flag_grp1] = df_filtered[flag_grp1]["conf_grp2"] / df_filtered[flag_grp1]["lift_grp2"]
        df_filtered["p_x1"][flag_grp1] = 0
        df_filtered["p_x2"][flag_grp1] = df_filtered[flag_grp1]["supp_grp2"] / df_filtered[flag_grp1]["conf_grp2"]

        flag_grp2 = df_filtered["idx_grp2"] == 0
        df_filtered["p_y1"][flag_grp2] = df_filtered[flag_grp2]["conf_grp1"] / df_filtered[flag_grp2]["lift_grp1"]
        df_filtered["p_y2"][flag_grp2] = 0
        df_filtered["p_x1"][flag_grp2] = df_filtered[flag_grp2]["supp_grp1"] / df_filtered[flag_grp2]["conf_grp1"]
        df_filtered["p_x2"][flag_grp2] = 0

        X_len = df_filtered["X"].apply(lambda x : len(ast.literal_eval(x)))
        df_filtered["delta_p_x"] = df_filtered["p_x1"] - df_filtered["p_x2"]
        delta_supp_1 = 2 * df_filtered["supp_diff"] / (df_filtered["p_y1"] + df_filtered["p_y2"])
        delta_supp_2 = (df_filtered["p_y2"] - df_filtered["p_y1"]) * (df_filtered["supp_grp1"] + df_filtered["supp_grp2"]) / (df_filtered["p_y1"] + df_filtered["p_y2"])
        df_filtered["supp_delta"] = delta_supp_1 + delta_supp_2
        df_filtered["new_three_weight"] = np.sign(df_filtered["delta_p_x"]) * np.sign(df_filtered["conf_diff"]) *\
                                          np.power(X_len,w1) * \
                                          np.power(np.abs(df_filtered["delta_p_x"]),w2) * \
                                          np.power(np.abs(df_filtered["conf_diff"]),w3)
        ruleset1 = deepcopy(df_filtered.sort_values(by = ["new_three_weight"], ascending=False).head(top_k))

        ruleset_asso_nostraight = pd.concat([ruleset1])
        ruleset_asso_nostraight_ = ruleset_asso_nostraight.drop_duplicates()
        print("remove {} duplicates".format(ruleset_asso_nostraight_.shape[0] - ruleset_asso_nostraight.shape[0]))
        rulesets_asso_nostraight[wk][epi] = deepcopy(ruleset_asso_nostraight_)
        print("final shape", ruleset_asso_nostraight.shape)

        ruleset_asso_nostraight.to_csv(focus_path + "SPMF_select/" + "asso_rule_nostraight" + "_" + wk + "_" + epi + ".csv", index = False)